#### Importación de librerías

In [ ]:
import numpy as np
import torch

#### Clase Conv2D

Se crea una clase llamada ***Conv2D*** que al instanciarla se le pasa el número de filtros, canales de entrada, tamaño del filtro (se asume cuadrado), stride y tipo de padding. La clase posee 3 métodos: ***forward***, ***backward df*** y ***backward dx***.

El método ***forward*** recibe como argumento la matriz de entrada (layer input) y devuelve el resultado de la convolución con el filtro. Los métodos ***backward df*** y ***backward dx*** reciben como argumento la derivada de la función de costo respecto a la salida de la convolución $\frac{\partial{L}}{\partial{O}}$ y retornan $\frac{\partial{L}}{\partial{f}}$ y $\frac{\partial{L}}{\partial{X}}$, respectivamente.

Dentro de la clase se generan funciones auxiliares para implementar algunos cálculos que se utilzan dentro de los métodos mencionados.

Para el padding hay 3 funciones: ***same***, ***valid*** y ***full***. El ***full_padding*** se emplea para calcular la convolución "completa" para hallar $\frac{\partial{L}}{\partial{X}}$. Para esta convolución es necesario rotar el filtro en 180° por medio de la función ***flip180***. Para calcular $\frac{\partial{L}}{\partial{f}}$ y $\frac{\partial{L}}{\partial{X}}$ cuando el stride es mayor que 1, utilizo la funcion ***dilation*** para insertar filas y columnas de ceros ($stride - 1$). La función ***convolve*** la saqué de la notebook dada en clase. 

In [ ]:
class Conv2D():
  def __init__(self, filters, input_channels, filter_size, stride, pad = 'same'):
    # Inicializacion aleatoria de parámetros de los filtros
    self.W = np.random.randn(filters, input_channels, filter_size, filter_size)
    self.b = np.random.randn(filters)
    self.padding = pad
    self.stride = stride
    
    self.last_input = None
    self.last_input_padded = None

  @staticmethod
  def same_padding(n_H_prev, n_W_prev, filter_size, stride):
    """
    Cuando el padding es 'same', el stride debe ser obligatoriamente igual a 1
    """   
    pad = int((filter_size - 1)/2)
    n_H = int((n_H_prev + 2*pad - filter_size)/stride + 1)
    n_W = int((n_W_prev + 2*pad - filter_size)/stride + 1)
    return n_H, n_W, pad

  @staticmethod
  def valid_padding(n_H_prev, n_W_prev, filter_size, stride):
    n_H = int((n_H_prev - filter_size)/stride + 1)
    n_W = int((n_W_prev - filter_size)/stride + 1)
    return n_H, n_W

  @staticmethod
  def full_padding(X, f_size, n_H_dil, n_H_last_input):
    pad = int((n_H_last_input - 1 - n_H_dil + f_size) / 2)
    X_full_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0))
    return X_full_padded

  @staticmethod
  def flip180(W):
    W_flip = W.reshape(W.size)
    W_flip = W_flip[::-1]
    W_flip = W_flip.reshape(W.shape)
    return W_flip

  # Función para realizar la operación de convolución
  @staticmethod
  def convolve(X, W, b = 0):
    """
    Argumentos:
      X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
      W: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
      b: Entero con el valor de bias de la capa actual
    Retorna:
      Z: Entero con el valor del resultado
    """
    # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
    aux = X * W
    # Realizo la suma de todos los elementos
    aux = np.sum(aux)
    # Le sumo el valor del bias para obtener Z
    Z = aux + float(b)
    return Z
  
  @staticmethod
  def dilation(X, stride):
    m = X.shape[2]
    for j in range(stride - 1):
      X = np.insert(X, [i for i in range(1, m + j*(m-1), j+1)], 0, axis = 2)
      X = np.insert(X, [i for i in range(1, m + j*(m-1), j+1)], 0, axis = 3)
    return X
  
  def conv_forward(self, layer_input):
    """
    Argumentos:
      layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev)
      W: Array numpy con los pesos de los filtros utilizados en la capa actual (n_C, n_C_prev, filter_size, filter_size)
      b: Array numpy con los valores de bias utilizados en la capa actual (1, 1, 1, n_C)
      stride: Entero con el valor de stride utilizado en la capa actual.
      padding: 

    Retorna:
      Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_C, n_H, n_W)
    """
    # Guardo la entrada (última) padeada
    self.last_input = layer_input
    # Obtengo las dimensiones de la entrada
    (batch_size, n_C_prev, n_H_prev, n_W_prev) = layer_input.shape
    # Obtengo las dimensiones de los filtros
    (n_C, n_C_prev, filter_size, filter_size) = self.W.shape

    # Obtengo la entrada padeada de acuerdo al tipo de pad seleccionado
    if self.padding == 'same':
      n_H, n_W, pad = self.same_padding(n_H_prev, n_W_prev, filter_size, self.stride)
      layer_input_padded = np.pad(layer_input, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0))
    elif self.padding == 'valid':
      n_H, n_W = self.valid_padding(n_H_prev, n_W_prev, filter_size, self.stride)
      layer_input_padded = layer_input

    # Guardo la entrada (última) padeada
    self.last_input_padded = layer_input_padded
    # Inicializo el volumen de salida con ceros
    Z = np.zeros([batch_size, n_C, n_H, n_W])

    # Comienzo iterando sobre cada ejemplo del batch
    for i in range(batch_size):
      # Itero sobre el eje vertical del volumen de salida
      for h in range(n_H):
        # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        y_start = self.stride * h
        y_end = y_start + filter_size
        # Itero sobre el eje horizontal del volumen de salida
        for w in range(n_W):
          # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
          x_start = self.stride * w
          x_end = x_start + filter_size
          # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
          slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
          # Itero sobre la cantidad de canales del volumen de salida
          for c in range(n_C):
            # Obtengo el valor del filtro y bias del canal correspondiente
            filter = self.W[c, :, :, :]
            bias = self.b[c]
            # Computo la operación de convolución para esta ventana
            Z[i, c, h, w] = self.convolve(slice_from_input_padded, filter, bias)

    return Z

  def conv_forward_pytorch(self, layer_input):
    """
    Esta función sólo se agrega para comparar la función forward implementada 'a mano' con Pytorch
    """
    return torch.nn.functional.conv2d(torch.tensor(layer_input), torch.tensor(self.W), torch.tensor(self.b), stride = self.stride, padding = self.padding)

  def conv_backward_df(self, d_layer_output):
    # Calculo dL/df como la convolución entre la dL/dO que se le pasa al método backward y la entrada padeada
    
    # Obtengo las dimensiones de la entrada padeada
    (batch_size, n_C_prev, n_H_prev, n_W_prev) = self.last_input_padded.shape

    # Obtengo las dimensiones de la dL/dO con dilatación, si stride es 1 es directamente d_layer_output
    d_layer_output_dil = self.dilation(d_layer_output, self.stride)
    (batch_size_output, n_C, f_size, f_size) = d_layer_output_dil.shape

    # Se considera stride = 1 para las dimensiones de salida, ya que se contempla en la dilatación
    stride = 1
    n_H = int((n_H_prev - f_size)/stride + 1)
    n_W = int((n_W_prev - f_size)/stride + 1)
    # Inicializo el volumen de salida con ceros
    df = np.zeros([n_C, n_C_prev, n_H, n_W])

    # Comienzo iterando sobre numero de canales de entrada
    for n in range(n_C_prev):
      # Itero sobre el eje vertical del volumen de salida
      for h in range(n_H):
        # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        y_start = stride * h
        y_end = y_start + f_size
        # Itero sobre el eje horizontal del volumen de salida
        for w in range(n_W):
          # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
          x_start = stride * w
          x_end = x_start + f_size
          # Itero sobre la cantidad de canales del volumen de salida
          for c in range(n_C):
            aux = 0
            for i in range(batch_size):
              # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
              slice_from_input_padded = self.last_input_padded[i, n, y_start:y_end, x_start:x_end]
               # Obtengo el valor del filtro del canal correspondiente
              filter = d_layer_output_dil[i, c, :, :]
              aux = aux + self.convolve(slice_from_input_padded, filter)
            
            # Computo la operación de convolución para esta ventana (media del batchsize)
            df[c, n, h, w] = aux / batch_size
    return df

  def conv_backward_dx(self, d_layer_output):
    
    # Rotación filtros de 180°
    (filters, input_channels, f_size, f_size) = self.W.shape
    W_flip = np.zeros(self.W.shape)
    for i in range(filters):
      for j in range(input_channels):
        W_flip [i,j,:,:]= self.flip180(self.W[i,j,:,:])
    
    # Aplico dilatación a dL/dO (si stride = 1 entonces dL/dO =d_layer_output) y luego full padding
    d_layer_output_dil = self.dilation(d_layer_output, self.stride)
    n_H_dil = d_layer_output_dil.shape[2]
    n_H_last_input = self.last_input.shape[2]
    d_layer_output_dilpad = self.full_padding(d_layer_output_dil, f_size, n_H_dil, n_H_last_input)
    (batch_size, filters, n_H_prev, n_W_prev) = d_layer_output_dilpad.shape

    # La convolución dL/dO * f se hace con stride = 1
    stride = 1
    # dimensiones de salida de la convolucion (igual a x)
    n_H = int(n_H_prev - f_size + 1)
    n_W = int(n_W_prev - f_size + 1)
    # Inicializo el volumen de salida con ceros
    dx = np.zeros([batch_size, input_channels, n_H, n_W])

    # Comienzo iterando sobre los batches
    for i in range(batch_size):
      # Itero sobre el eje vertical del volumen de salida
      for h in range(n_H):
        # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        y_start = stride * h
        y_end = y_start + f_size
        # Itero sobre el eje horizontal del volumen de salida
        for w in range(n_W):
          # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
          x_start = stride * w
          x_end = x_start + f_size
          # Itero sobre la cantidad de canales del volumen de salida
          for c in range(input_channels):
            aux = 0
            for n in range(filters):            
              # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
              slice_from_output_dilpad = d_layer_output_dilpad[i, n, y_start:y_end, x_start:x_end]
              # Obtengo el valor del filtro del canal correspondiente
              filter = W_flip[n, c, :, :]              
              aux = aux + self.convolve(slice_from_output_dilpad, filter)
            # Computo la operación de convolución para esta ventana
            dx[i, c, h, w] = aux
    return dx

#### Definición de parámetros

In [ ]:
np.random.seed(1234)

# Dimensiones de la entrada
batch_size = 10
input_height, input_width = (7, 7)
input_channels = 4

# Dimensiones de la convolucional
filters = 8
filter_size = 3
stride = 1
pad = 'same'

Defino aleatoriamente una input.

In [ ]:
# x test
test_array = np.random.randn(batch_size, input_channels, input_height, input_width)

#### Resultados

Instancio la clase y comparo resultados del método ***forward*** con *Pytorch* si no se cumple *pad = same* y *stride > 1*. 

In [ ]:
# Instancio la clase
conv = Conv2D(filters, input_channels, filter_size, stride, pad)

# Llamo al método forward
conv_forw = conv.conv_forward(test_array)
print("Convolución: Result shape: {}".format(conv_forw.shape))
print("Convolución: Result value: {}".format(conv_forw[1, 1, 1, 1]))

# Comparo con pytorch 
if not((pad == 'same') and (stride > 1)):
  conv_forw_pyt = conv.conv_forward_pytorch(test_array)
  assert(conv_forw.shape == conv_forw_pyt.shape)
  print("Convolución: Result shape: {}".format(conv_forw_pyt.shape))
  print("Convolución: Result value: {}".format(conv_forw_pyt[1, 1, 1, 1]))

Convolución: Result shape: (10, 8, 7, 7)
Convolución: Result value: 2.6830698172227043
Convolución: Result shape: torch.Size([10, 8, 7, 7])
Convolución: Result value: 2.6830698172227048


Defino aleatoriamente la $\frac{\partial{L}}{\partial{O}}$ y se la paso a los métodos ***backward df*** y ***backward dx***.

Verifico que las dimensiones de $\frac{\partial{L}}{\partial{f}}$ coincida con las dimensiones que definí para el filtro y que $\frac{\partial{L}}{\partial{X}}$ coincida con las dimensiones del input.

In [ ]:
test_d_output = np.random.randn(conv_forw.shape[0], conv_forw.shape[1], conv_forw.shape[2], conv_forw.shape[3])

conv_back_df = conv.conv_backward_df(test_d_output)

print("Convolución: Backward dL/df result shape: {}".format(conv_back_df.shape))
print("Convolución: Backward dL/df result value: {}".format(conv_back_df[1, 1, 1, 1]))

conv_back_dx = conv.conv_backward_dx(test_d_output)

print("Convolución: Backward dL/dx result shape: {}".format(conv_back_dx.shape))
print("Convolución: Backward dL/dx result value: {}".format(conv_back_dx[1, 1, 1, 1]))

Convolución: Backward dL/df result shape: (8, 4, 3, 3)
Convolución: Backward dL/df result value: -5.0571654466394875
Convolución: Backward dL/dx result shape: (10, 4, 7, 7)
Convolución: Backward dL/dx result value: -5.699161013117985
